In [1]:
!pip install datasets
!pip install gensim


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from datasets import load_dataset

/Users/adithyasamudrala/SC4002-NLP-Project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

In [4]:

print(test_dataset.to_pandas().head(15))

print(test_dataset[:15])  # View the first 5 rows as a dictionary


                                                 text  label
0   lovingly photographed in the manner of a golde...      1
1               consistently clever and suspenseful .      1
2   it's like a " big chill " reunion of the baade...      1
3   the story gives ample opportunity for large-sc...      1
4                   red dragon " never cuts corners .      1
5   fresnadillo has something serious to say about...      1
6   throws in enough clever and unexpected twists ...      1
7   weighty and ponderous but every bit as filling...      1
8   a real audience-pleaser that will strike a cho...      1
9   generates an enormous feeling of empathy for i...      1
10  exposing the ways we fool ourselves is one hou...      1
11  it's up to you to decide whether to admire the...      1
12  mostly , [goldbacher] just lets her complicate...      1
13  . . . quite good at providing some good old fa...      1
14  at its worst , the movie is pretty diverting ;...      1
{'text': ['lovingly phot

# Google Word2Vec

In [5]:
# from gensim.models import KeyedVectors

# # Path to the .bin file
# model_path = 'GoogleNews-vectors-negative300.bin'  # Update with the exact path to the .bin file

# # Load the Word2Vec model
# word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
# print("Model loaded successfully.")

In [8]:
import os
import gensim.downloader as api
from gensim.models import KeyedVectors

# Define the model path
model_name = "word2vec-google-news-300"

path = api.load(model_name, return_path=True)
print(f"Model downloaded and saved at: {path}")


# Load the model
word2vec_model = KeyedVectors.load_word2vec_format(path, binary=True)
print("Model loaded successfully.")


Model downloaded and saved at: /Users/adithyasamudrala/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
Model loaded successfully.


In [9]:
# Find words similar to "king"
similar_words = word2vec_model.most_similar("king", topn=10)
print("Words similar to 'king':")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

Words similar to 'king':
kings: 0.7138
queen: 0.6511
monarch: 0.6413
crown_prince: 0.6204
prince: 0.6160
sultan: 0.5865
ruler: 0.5798
princes: 0.5647
Prince_Paras: 0.5433
throne: 0.5422


In [ ]:
# Compute similarity between "king" and "queen"
similarity = word2vec_model.similarity("king", "queen")
print(f"Similarity between 'king' and 'queen': {similarity:.4f}")

In [12]:
# Get the vector for the word "apple"
vector = word2vec_model["an"]
print("Vector for 'apple':", vector)

Vector for 'apple': [ 0.12597656  0.19042969  0.06982422  0.07226562 -0.21875     0.02758789
  0.05395508 -0.07666016  0.04296875 -0.12988281  0.05151367 -0.10498047
 -0.11474609  0.05566406 -0.05029297 -0.00424194  0.08447266  0.00915527
 -0.09277344 -0.02770996  0.08935547 -0.11425781 -0.13476562 -0.09521484
 -0.10742188 -0.10888672 -0.02563477  0.31054688  0.17382812  0.19921875
  0.0090332   0.02905273 -0.09863281 -0.10058594 -0.21191406 -0.11962891
 -0.06542969  0.125      -0.07617188 -0.11181641 -0.19726562  0.12695312
  0.02770996  0.04785156  0.11035156  0.04760742  0.01330566  0.19726562
  0.11962891  0.04296875  0.07861328 -0.07128906  0.21972656  0.00772095
 -0.16210938  0.125       0.10498047 -0.03588867 -0.03710938 -0.11767578
 -0.29296875  0.11669922  0.04077148 -0.00152588 -0.06445312  0.24023438
  0.29101562 -0.03198242 -0.02160645  0.08398438  0.0703125   0.05175781
  0.17871094 -0.05639648  0.09472656 -0.01275635  0.02050781  0.13867188
 -0.00069427  0.26757812  0.174

# GLOVE Embedding Model

In [13]:
glove_model = api.load("glove-wiki-gigaword-300")
print("GloVe model loaded successfully.")

[======================----------------------------] 45.1% 169.5/376.1MB downloaded

In [ ]:
# Step 2: Testing the GloVe model

# 1. Check similar words
print("Words similar to 'king':")
similar_words = glove_model.most_similar("king", topn=10)
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

# 2. Compute word similarity
similarity = glove_model.similarity("king", "queen")
print(f"\nSimilarity between 'king' and 'queen': {similarity:.4f}")

# 3. Perform analogy
analogy_result = glove_model.most_similar(positive=["woman", "king"], negative=["man"], topn=1)
print("\nAnalogy result for 'king - man + woman':", analogy_result)

# 4. Check for a word in the vocabulary
word = "apple"
if word in glove_model:
    print(f"\n'{word}' is in the vocabulary.")
else:
    print(f"\n'{word}' is not in the vocabulary.")

# 5. Get word vector
vector = glove_model["apple"]
print("\nVector for 'apple':", vector)

# Q1a) What is the size of the vocabulary formed from your training data?

In [ ]:
import re
from collections import Counter

def tokenize(text):
    # Use simple whitespace tokenization and lowercasing
    return re.findall(r'\b\w+\b', text.lower())

# Count unique tokens
counter = Counter()
for example in train_dataset:
    tokens = tokenize(example["text"])
    counter.update(tokens)

# Size of the vocabulary
vocab_size = len(counter)
print("Vocabulary size:", vocab_size)

# Q1b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

In [ ]:
counter = 0
for example in train_dataset:
    tokens = tokenize(example["text"])
    tokens = [token.lower() for token in tokens]
    for token in tokens:
        if token not in glove_model:
            counter += 1
            print(token)

print("Number of OOV words:", counter)

# Q1c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.